<a href="https://colab.research.google.com/github/PeterMosier/photo-resizer-notebook/blob/main/ImageResizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload Files to a Colab temporary workspace

In [1]:
from google.colab import files
uploaded_files = files.upload()

Saving 1R0A2195.JPEG to 1R0A2195.JPEG
Saving 1R0A9368.JPEG to 1R0A9368.JPEG
Saving 1R0A9543.JPEG to 1R0A9543.JPEG


# Confirm the upload: list the files

In [2]:
import os

print("Uploaded files:")
for filename in uploaded_files.keys():
    print(filename)

Uploaded files:
1R0A2195.JPEG
1R0A9368.JPEG
1R0A9543.JPEG


# Install Necessary Libraries

In [3]:
!pip install pillow tqdm
!pip install exif

## Imports and Paths

In [4]:
import os
from tqdm import tqdm

# Source folder is Colab current working directory
SOURCE_FOLDER = "."
DEST_FOLDER = "./webp_resized"

# Set image size constraints
MAX_WIDTH = 1024
MAX_HEIGHT = 768

# Create destination folder if it doesn't exist
os.makedirs(DEST_FOLDER, exist_ok=True)

## Resize Function with Copyright Overlay

In [5]:
from PIL import Image, ImageDraw, ImageFont
from exif import Image as ExifImage
import json

def resize_image(image_path, output_path, copyright_text="© Bruce Graham Photography | UntamedFrames.ca"):
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        width, height = img.size

        # Calculate scale based on both width and height constraints
        width_scale = MAX_WIDTH / width
        height_scale = MAX_HEIGHT / height
        scale = min(width_scale, height_scale, 1.0)

        new_size = (int(width * scale), int(height * scale))
        resized = img.resize(new_size, Image.LANCZOS)

        # Draw copyright text
        draw = ImageDraw.Draw(resized)
        font_size = int(new_size[1] * 0.030)  # ~3.0% of image height

        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()

        # Measure text size
        bbox = draw.textbbox((0, 0), copyright_text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        # Bottom-right position
        x_br = new_size[0] - text_width - 10
        y_br = new_size[1] - text_height - 10

        # Top-left position
        x_tl = 10
        y_tl = 10

        # Bottom-right: shadow + main text
        draw.text((x_br + 1, y_br + 1), copyright_text, font=font, fill=(0, 0, 0, 100))
        draw.text((x_br, y_br), copyright_text, font=font, fill=(255, 255, 255, 180))

        # Top-left: shadow + main text
        draw.text((x_tl + 1, y_tl + 1), copyright_text, font=font, fill=(0, 0, 0, 100))
        draw.text((x_tl, y_tl), copyright_text, font=font, fill=(255, 255, 255, 100))

        # Save readable EXIF as .json (safe fallback for corrupt headers)
        try:
            with open(image_path, "rb") as f:
                exif_img = ExifImage(f)
                if exif_img.has_exif:
                    exif_dict = {}
                    for tag in exif_img.list_all():
                        try:
                            value = getattr(exif_img, tag)
                            exif_dict[tag] = str(value)
                        except (AttributeError, NotImplementedError, TypeError):
                            continue
                    sidecar_path = output_path.replace(".webp", ".json")
                    with open(sidecar_path, "w", encoding="utf-8") as f_out:
                        json.dump(exif_dict, f_out, indent=2)
        except Exception as e:
            print(f"EXIF skipped for {os.path.basename(image_path)}: {e}")

        # Save as WebP
        resized.save(output_path, "WEBP", quality=85)

## Run the Batch

In [6]:
for filename in uploaded_files.keys():
    input_path = os.path.join(SOURCE_FOLDER, filename)
    output_name = os.path.splitext(filename)[0] + ".webp"
    output_path = os.path.join(DEST_FOLDER, output_name)

    resize_image(input_path, output_path)
    print(f"Converted: {filename} → {output_name}")

Converted: 1R0A2195.JPEG → 1R0A2195.webp
Converted: 1R0A9368.JPEG → 1R0A9368.webp
Converted: 1R0A9543.JPEG → 1R0A9543.webp


# Zip and download

In [7]:
import shutil
from google.colab import files

# Create a zip archive of the entire output folder (includes .webp and .json files)
shutil.make_archive("webp_resized", "zip", "webp_resized")

# Download the zip file
files.download("webp_resized.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Clear Uploaded files and the output folder
An alternative to running the cell below, is reset this Colab Notbook environment via menu:
> **Runtime > Restart Runtime**

This resets everything, including erasing the files. Then you can re-run this notebook from the top.

In [18]:
import shutil
import os

# Delete all uploaded files
for filename in uploaded_files.keys():
    if os.path.exists(filename):
        os.remove(filename)

# Delete the output folder and zip (if they exist)
shutil.rmtree("webp_resized", ignore_errors=True)
if os.path.exists("webp_resized.zip"):
    os.remove("webp_resized.zip")

print("Workspace cleared. Ready for new uploads!")

Workspace cleared. Ready for new uploads!
